## Imports

In [13]:
#importing necessary libraries
from rajvi_mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

## Importing Coverage Function (allows me to pick any branch I want)

In [95]:
#reading in library branch and population demographics, both with geometry data
#branchInfo = pd.read_csv('../library neighborhoods/completedCyclingNeighborhoodRadiiDF.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc1branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc3minus1branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc5minus3branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc7minus5branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc9minus7branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc11minus9branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc13minus11branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc15minus13branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc17minus15branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc19minus17branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc21minus19branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc23minus21branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc25minus23branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/cyc27minus25branchRadii.csv')
branchInfo = pd.read_csv('../library neighborhoods/cyc30minus27branchRadii.csv')
censusTracts = pd.read_csv('../data/clean/population_demographics.csv')
censusDemographics = pd.read_csv('../data/dani data/census_demos.csv')

In [96]:
#removing unecessary columns from branchInfo
branchInfo = branchInfo.drop(columns = ['Unnamed: 0'])
#removing unecessary columns from censusTracts
censusTracts = censusTracts[['geoid','geometry','qualifying name']]

#converting censusTracts to a geodataframe (we could not load it as one since it already had a geometry column so we are simply
#converting the geometry column to a proper gpd geometry column).
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )

branchInfo[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchInfo['LOCATION']]
branchInfo.loc[:, 'LATITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LATITUDE'])
branchInfo.loc[:, 'LONGITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LONGITUDE'])

branchInfo = gpd.GeoDataFrame(
    branchInfo.loc[:, [c for c in branchInfo.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(branchInfo["geometry"]),
    crs="epsg:4326",
    )

In [38]:
censusPops = censusDemographics[['geoid', 'total population']].copy(deep=True)
censusPops

,geoid,total population
0,17031823303,4059
1,17031823304,6316
2,17031740400,4738
3,17031740300,5017
4,17031822802,3750
...,...,...
864,17031320400,3058
865,17031320101,7952
866,17031081403,8984
867,17031081402,5613


In [17]:
#helper method to ensure that two polygons overlap before getting the intersection. Gets the intersection then calculates
#the percent of overlapping area for a census tract and saves it to a dictionary with the census tract geo-id as the key and
#the percent as the value.
def check(polygon1, df, geoname, dictionary,key):
   
    for i in range(len(df)):
        if polygon1.intersects(df.loc[i,geoname]): 
            overlapPolygon = (polygon1.intersection(df.loc[i,geoname]))
            poly_area, poly_perimeter = geod.geometry_area_perimeter(overlapPolygon)
            overlapArea = poly_area*-1
            propOverlap = overlapArea / df.loc[i,'Area']
            #print(propOverlap)
            dictionary[df.loc[i, key]] = propOverlap
    return dictionary
   

In [18]:
#sets the crs of gpd types
def geoSetup(gdf):
    
    gdf = gdf.set_crs('EPSG:4326')
    

In [19]:
#cdf = gdf of geometry area which you want to get the coverage score
#idf = gdf of points for which you want to get an isochrone and dictionary of percent of overlap in cdf areas
#scoreString = a string that is the name of the coverage score column you choose
#dictString = a string that is the name of the dictionary column you choose
#cgs = a string that is the name of the geometry column in the cdf
#igs = a string that is the name of the geometry column in the idf
#lat = a string that is the name of the latitude column in the idf
#lon = a string that is the name of the longitude column in the idf
#key = a string that is the name of the column in the cdf that you want to represent the key of the dictionary

def appendADS(cdf,idf,scoreString,dictString,cgs,igs,lat,lon,key):
    
    #api = MapboxAPI()
    cdf[scoreString] = 0.0
    idf[dictString] = ''
    censusAreas = []
    global geod 
    geod = Geod(ellps='WGS84')
   
    geoSetup(cdf)
    geoSetup(idf)
   
    for index, row in cdf.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row[cgs])
        poly_area = poly_area*-1
        censusAreas.append(poly_area)
    
    cdf['Area'] = censusAreas



    for i in range(len(idf)):
        dictionary = {}
        dictionary = check(idf.loc[i,igs],cdf,cgs, dictionary,key)
        branchInfo.loc[i,dictString]= [dictionary]
    
        for i in range(len(cdf)):
   
            if dictionary.get(cdf.loc[i, key]) != None:
                score = cdf.loc[i, scoreString]
                cdf.loc[i, scoreString] = score + dictionary.get(cdf.loc[i, key])
  
   

In [97]:
appendADS(censusTracts,branchInfo, 'score', 'list of dict','geometry','geometry','LATITUDE','LONGITUDE','geoid')

/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt

In [11]:
branchInfo.head(3)

,BRANCH,LOCATION,LATITUDE,LONGITUDE,geometry,list of dict
0,Albany Park,"(41.97557881655979, -87.71361314512697)",41.975579,-87.713613,"POLYGON ((-87.71261 41.97775, -87.71361 41.978...",[{17031831800: 0.08322953535333169}]
1,Altgeld,"(41.65719847558056, -87.59883829075173)",41.657198,-87.598838,"POLYGON ((-87.59784 41.65899, -87.59984 41.658...","[{17031540101: 0.015409641217401857, 170315401..."
2,Archer Heights,"(41.80110836194246, -87.72648385568911)",41.801108,-87.726484,"POLYGON ((-87.72748 41.80338, -87.72845 41.802...","[{17031570500: 0.07641335104611449, 1703157030..."


## Average Travel Time Function

In [90]:

#Internet
def min5Radius(DemographicsVariables, branchWithCensus):
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    #Branch_pop_withInternet = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['Total']).sum() 
    #result = Branch_pop_withInternet/pop_in_branch
    return pop_in_branch   

#Income


In [91]:
def branch_demographics(branchWithCensus, nameOfBranch):
    branch_snapshot_df = pd.DataFrame({'Branch': [nameOfBranch]})
    list1 = ['5 Min']
    for i in list1:
        branch_snapshot_df[i] = min5Radius(i, branchWithCensus)
    
    return branch_snapshot_df

In [72]:
def incrementPopulations(branchDataset):
    popPerLib = pd.DataFrame(columns=['Branch', 'Given Min Pop'])

    for i in range(len(branchDataset)):
        
        #Locating only Altgeld's column 'list of dict'
        libraryDictList = branchDataset.loc[i,'list of dict']
        branchName = branchDataset.loc[i,'BRANCH']
        
        #Removed set of brackets, just a dictionary now
        finalDictionary = libraryDictList[0]

        #Seperating into two columns
        dictToList = pd.DataFrame(finalDictionary.items(), columns= ['geoid','Percent Overlap'])
        libraryAndCensus = censusPops.merge(dictToList, on='geoid')

        libraryAndCensus['counts'] = 0
        for j in range(len(libraryAndCensus)):
            libraryAndCensus.loc[j,'counts'] = libraryAndCensus.loc[j,'total population'] * libraryAndCensus.loc[j,'Percent Overlap']
        
        libPop = pd.DataFrame({'BRANCH': [branchName],'Given Min Pop': libraryAndCensus['counts'].sum()})
        popPerLib = pd.concat([popPerLib, libPop])
        
    popPerLib = popPerLib.drop(columns = ['Branch'])
    popPerLib = popPerLib.reset_index()
    popPerLib = popPerLib.drop(columns = ['index'])
    
    return popPerLib

In [98]:
oneRing = incrementPopulations(branchInfo)

In [99]:
c30Min27Population = oneRing.copy(deep = True)
c30Min27Population

,Given Min Pop,BRANCH
0,0,Albany Park
1,4610.718154,Altgeld
2,0,Archer Heights
3,0,Austin
4,0,Austin-Irving
...,...,...
76,9.573944,West Pullman
77,6.883607,West Town
78,6.772762,"Whitney M. Young, Jr."
79,0.018153,Woodson Regional Library


In [142]:
allRadiiPops = pd.DataFrame(columns=['Branch', 'Prop 0 to 1 Min','Prop 1 to 3 Min', 'Prop 3 to 5 Min', 'Prop 5 to 7 Min', 'Prop 7 to 9 Min', 'Prop 9 to 11 Min', 
                                     'Prop 11 to 13 Min', 'Prop 13 to 15 Min', 'Prop 15 to 17 Min', 'Prop 17 to 19 Min',
                                     'Prop 19 to 21 Min', 'Prop 21 to 23 Min', 'Prop 23 to 25 Min', 'Prop 25 to 27 Min', 
                                     'Prop 27 to 30 Min', 'Total Pop Around Library'])

In [143]:
allRadiiPops['Branch'] = c1Min0Population['BRANCH']
allRadiiPops['Prop 0 to 1 Min'] = c1Min0Population['Given Min Pop']
allRadiiPops['Prop 1 to 3 Min'] = c3Min1Population['Given Min Pop']
allRadiiPops['Prop 3 to 5 Min'] = c5Min3Population['Given Min Pop']
allRadiiPops['Prop 5 to 7 Min'] = c7Min5Population['Given Min Pop']
allRadiiPops['Prop 7 to 9 Min'] = c9Min7Population['Given Min Pop']
allRadiiPops['Prop 9 to 11 Min'] = c11Min9Population['Given Min Pop']
allRadiiPops['Prop 11 to 13 Min'] = c13Min11Population['Given Min Pop']
allRadiiPops['Prop 13 to 15 Min'] = c15Min13Population['Given Min Pop']
allRadiiPops['Prop 15 to 17 Min'] = c17Min15Population['Given Min Pop']
allRadiiPops['Prop 17 to 19 Min'] = c19Min17Population['Given Min Pop']
allRadiiPops['Prop 19 to 21 Min'] = c21Min19Population['Given Min Pop']
allRadiiPops['Prop 21 to 23 Min'] = c23Min21Population['Given Min Pop']
allRadiiPops['Prop 23 to 25 Min'] = c25Min23Population['Given Min Pop']
allRadiiPops['Prop 25 to 27 Min'] = c27Min25Population['Given Min Pop']
allRadiiPops['Prop 27 to 30 Min'] = c30Min27Population['Given Min Pop']
allRadiiPops['Total Pop Around Library'] = 0

In [144]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library
0,Albany Park,581.025386,5903.826978,18317.819933,7327.396596,1151.225511,1587.150802,4.483994,0,0,0,0,0,0,0,0,0
1,Altgeld,110.479828,746.066898,850.777436,896.620274,2055.557460,9716.220577,1753.674709,736.93711,378.370407,126.006702,2221.857316,33.854304,22.61998,16.085297,4610.718154,0
2,Archer Heights,839.082572,5533.932193,8078.425895,7673.739869,4828.072138,2779.257232,2075.134213,93.387157,148.364089,0.927795,0,0,0,0,0,0
3,Austin,755.416637,5341.262486,14398.756484,11404.807683,6984.739211,8085.196259,18153.234533,0.00198,0,0,0,0,0,0,0,0
4,Austin-Irving,808.238033,6779.573505,10199.223453,9897.954007,591.504884,0.000000,0.000000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,362.235430,3219.939290,7195.277264,10551.941898,17702.571115,4038.694407,213.491099,50.783464,16.963914,2.142894,5.631359,0.006999,0.161383,2.29509,9.573944,0
77,West Town,1554.425886,10409.107568,17049.245968,11469.158730,2904.934044,39.787123,0.000000,0,0,0,0,0,0,0,6.883607,0
78,"Whitney M. Young, Jr.",1127.654831,7422.247249,11275.637350,9798.770328,7113.588317,3185.705535,975.840297,0,0,0,0,0.012636,3.503324,2.134256,6.772762,0
79,Woodson Regional Library,327.765850,3754.555883,7881.365940,8565.977423,9400.401510,4743.373210,1199.843204,36.09633,0,0,0,0,0,0,0.018153,0


In [145]:
allRadiiPops = allRadiiPops.fillna(0)

In [146]:
allRadiiPops.head(4)

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library
0,Albany Park,581.025386,5903.826978,18317.819933,7327.396596,1151.225511,1587.150802,4.483994,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0
1,Altgeld,110.479828,746.066898,850.777436,896.620274,2055.557460,9716.220577,1753.674709,736.937110,378.370407,126.006702,2221.857316,33.854304,22.61998,16.085297,4610.718154,0
2,Archer Heights,839.082572,5533.932193,8078.425895,7673.739869,4828.072138,2779.257232,2075.134213,93.387157,148.364089,0.927795,0.000000,0.000000,0.00000,0.000000,0.000000,0
3,Austin,755.416637,5341.262486,14398.756484,11404.807683,6984.739211,8085.196259,18153.234533,0.001980,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0


In [147]:
allRadiiPops = allRadiiPops.set_index('Branch')

In [148]:
allRadiiPops['Total Pop Around Library'] = allRadiiPops.sum(axis=1)

In [149]:
allRadiiPops

,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library
Branch,,,,,,,,,,,,,,,,
Albany Park,581.025386,5903.826978,18317.819933,7327.396596,1151.225511,1587.150802,4.483994,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34872.929201
Altgeld,110.479828,746.066898,850.777436,896.620274,2055.557460,9716.220577,1753.674709,736.937110,378.370407,126.006702,2221.857316,33.854304,22.619980,16.085297,4610.718154,24275.846449
Archer Heights,839.082572,5533.932193,8078.425895,7673.739869,4828.072138,2779.257232,2075.134213,93.387157,148.364089,0.927795,0.000000,0.000000,0.000000,0.000000,0.000000,32050.323154
Austin,755.416637,5341.262486,14398.756484,11404.807683,6984.739211,8085.196259,18153.234533,0.001980,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,65123.415272
Austin-Irving,808.238033,6779.573505,10199.223453,9897.954007,591.504884,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28276.493882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Pullman,362.235430,3219.939290,7195.277264,10551.941898,17702.571115,4038.694407,213.491099,50.783464,16.963914,2.142894,5.631359,0.006999,0.161383,2.295090,9.573944,43371.709548
West Town,1554.425886,10409.107568,17049.245968,11469.158730,2904.934044,39.787123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.883607,43433.542926
"Whitney M. Young, Jr.",1127.654831,7422.247249,11275.637350,9798.770328,7113.588317,3185.705535,975.840297,0.000000,0.000000,0.000000,0.000000,0.012636,3.503324,2.134256,6.772762,40911.866884


In [150]:
allRadiiPops.reset_index(inplace = True)

In [151]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library
0,Albany Park,581.025386,5903.826978,18317.819933,7327.396596,1151.225511,1587.150802,4.483994,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34872.929201
1,Altgeld,110.479828,746.066898,850.777436,896.620274,2055.557460,9716.220577,1753.674709,736.937110,378.370407,126.006702,2221.857316,33.854304,22.619980,16.085297,4610.718154,24275.846449
2,Archer Heights,839.082572,5533.932193,8078.425895,7673.739869,4828.072138,2779.257232,2075.134213,93.387157,148.364089,0.927795,0.000000,0.000000,0.000000,0.000000,0.000000,32050.323154
3,Austin,755.416637,5341.262486,14398.756484,11404.807683,6984.739211,8085.196259,18153.234533,0.001980,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,65123.415272
4,Austin-Irving,808.238033,6779.573505,10199.223453,9897.954007,591.504884,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28276.493882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,362.235430,3219.939290,7195.277264,10551.941898,17702.571115,4038.694407,213.491099,50.783464,16.963914,2.142894,5.631359,0.006999,0.161383,2.295090,9.573944,43371.709548
77,West Town,1554.425886,10409.107568,17049.245968,11469.158730,2904.934044,39.787123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.883607,43433.542926
78,"Whitney M. Young, Jr.",1127.654831,7422.247249,11275.637350,9798.770328,7113.588317,3185.705535,975.840297,0.000000,0.000000,0.000000,0.000000,0.012636,3.503324,2.134256,6.772762,40911.866884
79,Woodson Regional Library,327.765850,3754.555883,7881.365940,8565.977423,9400.401510,4743.373210,1199.843204,36.096330,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018153,35909.397503


In [152]:
allRadiiPops['Prop 0 to 1 Min'] = allRadiiPops['Prop 0 to 1 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 1 to 3 Min'] = allRadiiPops['Prop 1 to 3 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 3 to 5 Min'] = allRadiiPops['Prop 3 to 5 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 5 to 7 Min'] = allRadiiPops['Prop 5 to 7 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 7 to 9 Min'] = allRadiiPops['Prop 7 to 9 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 9 to 11 Min'] = allRadiiPops['Prop 9 to 11 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 11 to 13 Min'] = allRadiiPops['Prop 11 to 13 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 13 to 15 Min'] = allRadiiPops['Prop 13 to 15 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 15 to 17 Min'] = allRadiiPops['Prop 15 to 17 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 17 to 19 Min'] = allRadiiPops['Prop 17 to 19 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 19 to 21 Min'] = allRadiiPops['Prop 19 to 21 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 21 to 23 Min'] = allRadiiPops['Prop 21 to 23 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 23 to 25 Min'] = allRadiiPops['Prop 23 to 25 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 25 to 27 Min'] = allRadiiPops['Prop 25 to 27 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 27 to 30 Min'] = allRadiiPops['Prop 27 to 30 Min']/allRadiiPops['Total Pop Around Library']

In [153]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library
0,Albany Park,0.016661,0.169295,0.525273,0.210117,0.033012,0.045512,0.000129,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,34872.929201
1,Altgeld,0.004551,0.030733,0.035046,0.036935,0.084675,0.400242,0.072239,3.035680e-02,0.015586,0.005191,0.091525,1.394567e-03,0.000932,0.000663,1.899303e-01,24275.846449
2,Archer Heights,0.026180,0.172664,0.252054,0.239428,0.150640,0.086715,0.064746,2.913766e-03,0.004629,0.000029,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,32050.323154
3,Austin,0.011600,0.082018,0.221100,0.175126,0.107254,0.124152,0.278751,3.039719e-08,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,65123.415272
4,Austin-Irving,0.028583,0.239760,0.360696,0.350042,0.020919,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,28276.493882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.008352,0.074241,0.165898,0.243291,0.408159,0.093118,0.004922,1.170889e-03,0.000391,0.000049,0.000130,1.613832e-07,0.000004,0.000053,2.207417e-04,43371.709548
77,West Town,0.035789,0.239656,0.392536,0.264062,0.066882,0.000916,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.584860e-04,43433.542926
78,"Whitney M. Young, Jr.",0.027563,0.181420,0.275608,0.239509,0.173876,0.077868,0.023852,0.000000e+00,0.000000,0.000000,0.000000,3.088585e-07,0.000086,0.000052,1.655452e-04,40911.866884
79,Woodson Regional Library,0.009128,0.104556,0.219479,0.238544,0.261781,0.132093,0.033413,1.005206e-03,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,5.055260e-07,35909.397503


In [154]:
allRadiiPops['Weighted Average Travel Time'] = 0

In [155]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.016661,0.169295,0.525273,0.210117,0.033012,0.045512,0.000129,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,34872.929201,0
1,Altgeld,0.004551,0.030733,0.035046,0.036935,0.084675,0.400242,0.072239,3.035680e-02,0.015586,0.005191,0.091525,1.394567e-03,0.000932,0.000663,1.899303e-01,24275.846449,0
2,Archer Heights,0.026180,0.172664,0.252054,0.239428,0.150640,0.086715,0.064746,2.913766e-03,0.004629,0.000029,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,32050.323154,0
3,Austin,0.011600,0.082018,0.221100,0.175126,0.107254,0.124152,0.278751,3.039719e-08,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,65123.415272,0
4,Austin-Irving,0.028583,0.239760,0.360696,0.350042,0.020919,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,28276.493882,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.008352,0.074241,0.165898,0.243291,0.408159,0.093118,0.004922,1.170889e-03,0.000391,0.000049,0.000130,1.613832e-07,0.000004,0.000053,2.207417e-04,43371.709548,0
77,West Town,0.035789,0.239656,0.392536,0.264062,0.066882,0.000916,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.584860e-04,43433.542926,0
78,"Whitney M. Young, Jr.",0.027563,0.181420,0.275608,0.239509,0.173876,0.077868,0.023852,0.000000e+00,0.000000,0.000000,0.000000,3.088585e-07,0.000086,0.000052,1.655452e-04,40911.866884,0
79,Woodson Regional Library,0.009128,0.104556,0.219479,0.238544,0.261781,0.132093,0.033413,1.005206e-03,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,5.055260e-07,35909.397503,0


In [158]:
for i in range(len(allRadiiPops)):
    allRadiiPops.loc[i,'Weighted Average Travel Time'] = (((allRadiiPops.loc[i,'Prop 0 to 1 Min'])*1)+
                                                        ((allRadiiPops.loc[i,'Prop 1 to 3 Min'])*2) +
                                                        ((allRadiiPops.loc[i,'Prop 3 to 5 Min'])*4) +
                                                        ((allRadiiPops.loc[i,'Prop 5 to 7 Min'])*6) +
                                                        ((allRadiiPops.loc[i,'Prop 7 to 9 Min'])*8) +
                                                        ((allRadiiPops.loc[i,'Prop 9 to 11 Min'])*10) +
                                                        ((allRadiiPops.loc[i,'Prop 11 to 13 Min'])*12) +
                                                        ((allRadiiPops.loc[i,'Prop 13 to 15 Min'])*14) +
                                                        ((allRadiiPops.loc[i,'Prop 15 to 17 Min'])*16) +
                                                        ((allRadiiPops.loc[i,'Prop 17 to 19 Min'])*18) +
                                                        ((allRadiiPops.loc[i,'Prop 19 to 21 Min'])*20) +
                                                        ((allRadiiPops.loc[i,'Prop 21 to 23 Min'])*22) +
                                                        ((allRadiiPops.loc[i,'Prop 23 to 25 Min'])*24) +
                                                        ((allRadiiPops.loc[i,'Prop 25 to 27 Min'])*26) +
                                                         (allRadiiPops.loc[i,'Prop 27 to 30 Min'])*28.5)

In [159]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.016661,0.169295,0.525273,0.210117,0.033012,0.045512,0.000129,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,34872.929201,4.437811
1,Altgeld,0.004551,0.030733,0.035046,0.036935,0.084675,0.400242,0.072239,3.035680e-02,0.015586,0.005191,0.091525,1.394567e-03,0.000932,0.000663,1.899303e-01,24275.846449,14.056106
2,Archer Heights,0.026180,0.172664,0.252054,0.239428,0.150640,0.086715,0.064746,2.913766e-03,0.004629,0.000029,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,32050.323154,5.780903
3,Austin,0.011600,0.082018,0.221100,0.175126,0.107254,0.124152,0.278751,3.039719e-08,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,65123.415272,7.555355
4,Austin-Irving,0.028583,0.239760,0.360696,0.350042,0.020919,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,28276.493882,4.218488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.008352,0.074241,0.165898,0.243291,0.408159,0.093118,0.004922,1.170889e-03,0.000391,0.000049,0.000130,1.613832e-07,0.000004,0.000053,2.207417e-04,43371.709548,6.569592
77,West Town,0.035789,0.239656,0.392536,0.264062,0.066882,0.000916,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.584860e-04,43433.542926,4.218355
78,"Whitney M. Young, Jr.",0.027563,0.181420,0.275608,0.239509,0.173876,0.077868,0.023852,0.000000e+00,0.000000,0.000000,0.000000,3.088585e-07,0.000086,0.000052,1.655452e-04,40911.866884,5.393937
79,Woodson Regional Library,0.009128,0.104556,0.219479,0.238544,0.261781,0.132093,0.033413,1.005206e-03,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,5.055260e-07,35909.397503,6.357643


In [160]:
allRadiiPops.sort_values(by=['Weighted Average Travel Time'], ascending=True)

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library,Weighted Average Travel Time
42,Little Italy,0.101263,0.426783,0.323930,0.111176,0.036839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000010,14000.814770,3.212593
15,Chicago Bee,0.054525,0.308844,0.449871,0.179701,0.000000,0.000088,0.001157,0.001006,0.001101,0.000409,0.000669,0.000143,0.000531,0.000359,0.001594,10208.501774,3.687781
12,Bucktown-Wicker Park,0.029622,0.255483,0.550147,0.161257,0.003491,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32695.620400,3.736647
8,Bezazian,0.036906,0.292021,0.437973,0.225622,0.007079,0.000021,0.000105,0.000150,0.000049,0.000034,0.000002,0.000000,0.000000,0.000000,0.000039,37759.186728,3.789295
6,Back of the Yards,0.028463,0.260037,0.519064,0.183624,0.006304,0.002507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23947.007447,3.802045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Dunning,0.013909,0.105531,0.243496,0.194526,0.066578,0.097516,0.025966,0.010112,0.119500,0.003225,0.001364,0.000000,0.064015,0.053020,0.001242,67443.040654,9.274675
55,Oriole Park,0.004976,0.049793,0.165158,0.156413,0.122394,0.156329,0.153946,0.032077,0.073150,0.013757,0.004665,0.015532,0.003947,0.031519,0.016343,48508.434626,9.775590
36,Jeffery Manor,0.010754,0.070430,0.079307,0.105801,0.074021,0.090754,0.179359,0.174096,0.107615,0.064971,0.002877,0.010538,0.000401,0.028386,0.000690,31418.268830,11.141016
1,Altgeld,0.004551,0.030733,0.035046,0.036935,0.084675,0.400242,0.072239,0.030357,0.015586,0.005191,0.091525,0.001395,0.000932,0.000663,0.189930,24275.846449,14.056106


In [161]:
allRadiiPops

,Branch,Prop 0 to 1 Min,Prop 1 to 3 Min,Prop 3 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 30 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.016661,0.169295,0.525273,0.210117,0.033012,0.045512,0.000129,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,34872.929201,4.437811
1,Altgeld,0.004551,0.030733,0.035046,0.036935,0.084675,0.400242,0.072239,3.035680e-02,0.015586,0.005191,0.091525,1.394567e-03,0.000932,0.000663,1.899303e-01,24275.846449,14.056106
2,Archer Heights,0.026180,0.172664,0.252054,0.239428,0.150640,0.086715,0.064746,2.913766e-03,0.004629,0.000029,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,32050.323154,5.780903
3,Austin,0.011600,0.082018,0.221100,0.175126,0.107254,0.124152,0.278751,3.039719e-08,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,65123.415272,7.555355
4,Austin-Irving,0.028583,0.239760,0.360696,0.350042,0.020919,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,28276.493882,4.218488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.008352,0.074241,0.165898,0.243291,0.408159,0.093118,0.004922,1.170889e-03,0.000391,0.000049,0.000130,1.613832e-07,0.000004,0.000053,2.207417e-04,43371.709548,6.569592
77,West Town,0.035789,0.239656,0.392536,0.264062,0.066882,0.000916,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.584860e-04,43433.542926,4.218355
78,"Whitney M. Young, Jr.",0.027563,0.181420,0.275608,0.239509,0.173876,0.077868,0.023852,0.000000e+00,0.000000,0.000000,0.000000,3.088585e-07,0.000086,0.000052,1.655452e-04,40911.866884,5.393937
79,Woodson Regional Library,0.009128,0.104556,0.219479,0.238544,0.261781,0.132093,0.033413,1.005206e-03,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,5.055260e-07,35909.397503,6.357643


In [164]:
averageCyclingTimeNeighborhoods = pd.DataFrame(columns=['Branch', 'Average Cycling Time (Minutes)'])


In [165]:
averageCyclingTimeNeighborhoods['Branch'] = allRadiiPops['Branch']
averageCyclingTimeNeighborhoods['Average Cycling Time (Minutes)'] = allRadiiPops['Weighted Average Travel Time']

In [166]:
averageCyclingTimeNeighborhoods

,Branch,Average Cycling Time (Minutes)
0,Albany Park,4.437811
1,Altgeld,14.056106
2,Archer Heights,5.780903
3,Austin,7.555355
4,Austin-Irving,4.218488
...,...,...
76,West Pullman,6.569592
77,West Town,4.218355
78,"Whitney M. Young, Jr.",5.393937
79,Woodson Regional Library,6.357643


In [170]:
averageCyclingTimeNeighborhoods.sort_values(by=['Average Cycling Time (Minutes)'], ascending=True)

,Branch,Average Cycling Time (Minutes)
42,Little Italy,3.212593
15,Chicago Bee,3.687781
12,Bucktown-Wicker Park,3.736647
8,Bezazian,3.789295
6,Back of the Yards,3.802045
...,...,...
23,Dunning,9.274675
55,Oriole Park,9.775590
36,Jeffery Manor,11.141016
1,Altgeld,14.056106


In [167]:
averageCyclingTimeNeighborhoods.to_csv('../library neighborhoods/averageCyclingTimeNeighborhoods.csv')